In [1]:
import numpy as np
import librosa

In [2]:
def wav2mfcc(file_path, max_pad_len=11):
    wave, sr=librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)
    pad_width = max_pad_len - mfcc.shape[1]
    mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfcc

In [3]:
import os

DATA_PATH = "./data_section_5.3/"
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)

In [4]:
def save_data_to_array(path=DATA_PATH, max_pad_len=11): 
    labels, _, _ = get_labels(path) 

    for label in labels: 
        mfcc_vectors = [] 
        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)] 
        for wavfile in wavfiles: 
            mfcc = wav2mfcc(wavfile, max_pad_len=max_pad_len) 
            mfcc_vectors.append(mfcc) 
        np.save(label + '.npy', mfcc_vectors)

In [5]:
from sklearn.model_selection import train_test_split

def get_train_test(split_ratio=0.6, random_state=42):
    # Here we'll get the available labels
    labels, indices, _ = get_labels(DATA_PATH)
    # Now we'll get the first arrays
    X = np.load(labels[0] + '.npy')
    y = np.zeros(X.shape[0])
    # Now we'll append all of the dataset into a single array
    for i, label in enumerate(labels[1:]):
        x = np.load(label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value=(i+1)))
    assert X.shape[0] == len(y)
    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

Using TensorFlow backend.


In [7]:
save_data_to_array()

In [8]:
X_train, X_test, y_train, y_test = get_train_test()

In [9]:
X_train = X_train.reshape(X_train.shape[0], 20, 11, 1)
X_test = X_test.reshape(X_test.shape[0], 20, 11, 1)
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

In [10]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2,2), activation='relu', input_shape=(20, 11, 1)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(3, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                 optimizer=keras.optimizers.Adadelta(),
                 metrics=['accuracy'])
    return model

In [11]:
model = get_model()

In [12]:
model.fit(X_train, y_train_hot, batch_size=100, epochs=200, verbose=1, validation_data=(X_test, y_test_hot))

Train on 3137 samples, validate on 2092 samples
Epoch 1/200
3137/3137 [==============================] - 0s - loss: 8.2217 - acc: 0.3542 - val_loss: 5.0519 - val_acc: 0.4570
Epoch 2/200
3137/3137 [==============================] - 0s - loss: 6.4119 - acc: 0.4329 - val_loss: 4.2759 - val_acc: 0.5196
Epoch 3/200
3137/3137 [==============================] - 0s - loss: 5.2519 - acc: 0.5046 - val_loss: 3.2836 - val_acc: 0.6042
Epoch 4/200
3137/3137 [==============================] - 0s - loss: 4.1817 - acc: 0.5566 - val_loss: 2.0793 - val_acc: 0.7161
Epoch 5/200
3137/3137 [==============================] - 0s - loss: 3.4772 - acc: 0.6191 - val_loss: 1.8815 - val_acc: 0.7323
Epoch 6/200
3137/3137 [==============================] - 0s - loss: 3.1679 - acc: 0.6379 - val_loss: 1.6621 - val_acc: 0.7424
Epoch 7/200
3137/3137 [==============================] - 0s - loss: 2.6766 - acc: 0.6774 - val_loss: 1.9176 - val_acc: 0.7323
Epoch 8/200
3137/3137 [==============================] - 0s - loss: 2.

3137/3137 [==============================] - 0s - loss: 0.3524 - acc: 0.8645 - val_loss: 0.3777 - val_acc: 0.8576
Epoch 66/200
3137/3137 [==============================] - 0s - loss: 0.3213 - acc: 0.8785 - val_loss: 0.3203 - val_acc: 0.8862
Epoch 67/200
3137/3137 [==============================] - 0s - loss: 0.3356 - acc: 0.8693 - val_loss: 0.2798 - val_acc: 0.9015
Epoch 68/200
3137/3137 [==============================] - 0s - loss: 0.3219 - acc: 0.8773 - val_loss: 0.3814 - val_acc: 0.8700
Epoch 69/200
3137/3137 [==============================] - 0s - loss: 0.3397 - acc: 0.8725 - val_loss: 0.3283 - val_acc: 0.8858
Epoch 70/200
3137/3137 [==============================] - 0s - loss: 0.3148 - acc: 0.8798 - val_loss: 0.2749 - val_acc: 0.9044
Epoch 71/200
3137/3137 [==============================] - 0s - loss: 0.3386 - acc: 0.8719 - val_loss: 0.3053 - val_acc: 0.8891
Epoch 72/200
3137/3137 [==============================] - 0s - loss: 0.3122 - acc: 0.8852 - val_loss: 0.3271 - val_acc: 0.88

3137/3137 [==============================] - 0s - loss: 0.2711 - acc: 0.8926 - val_loss: 0.2509 - val_acc: 0.9135
Epoch 130/200
3137/3137 [==============================] - 0s - loss: 0.2691 - acc: 0.8974 - val_loss: 0.2566 - val_acc: 0.9111
Epoch 131/200
3137/3137 [==============================] - 0s - loss: 0.2665 - acc: 0.8980 - val_loss: 0.2567 - val_acc: 0.9140
Epoch 132/200
3137/3137 [==============================] - 0s - loss: 0.2697 - acc: 0.8980 - val_loss: 0.2567 - val_acc: 0.9101
Epoch 133/200
3137/3137 [==============================] - 0s - loss: 0.2639 - acc: 0.8974 - val_loss: 0.2756 - val_acc: 0.9006
Epoch 134/200
3137/3137 [==============================] - 0s - loss: 0.2586 - acc: 0.8999 - val_loss: 0.2936 - val_acc: 0.8972
Epoch 135/200
3137/3137 [==============================] - 0s - loss: 0.2722 - acc: 0.8948 - val_loss: 0.2679 - val_acc: 0.9054
Epoch 136/200
3137/3137 [==============================] - 0s - loss: 0.2670 - acc: 0.8983 - val_loss: 0.2547 - val_ac

3137/3137 [==============================] - 0s - loss: 0.2588 - acc: 0.8961 - val_loss: 0.2512 - val_acc: 0.9101
Epoch 194/200
3137/3137 [==============================] - 0s - loss: 0.2639 - acc: 0.8999 - val_loss: 0.2489 - val_acc: 0.9116
Epoch 195/200
3137/3137 [==============================] - 0s - loss: 0.2596 - acc: 0.8977 - val_loss: 0.2639 - val_acc: 0.9044
Epoch 196/200
3137/3137 [==============================] - 0s - loss: 0.2419 - acc: 0.9012 - val_loss: 0.2577 - val_acc: 0.9082
Epoch 197/200
3137/3137 [==============================] - 0s - loss: 0.2303 - acc: 0.9127 - val_loss: 0.2833 - val_acc: 0.8920
Epoch 198/200
3137/3137 [==============================] - 0s - loss: 0.2408 - acc: 0.9056 - val_loss: 0.2642 - val_acc: 0.9082
Epoch 199/200
3137/3137 [==============================] - 0s - loss: 0.2387 - acc: 0.9076 - val_loss: 0.2714 - val_acc: 0.9097
Epoch 200/200
3137/3137 [==============================] - 0s - loss: 0.2523 - acc: 0.9060 - val_loss: 0.2554 - val_ac

In [13]:
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, 20, 11, 1)
    return get_labels()[0][np.argmax(model.predict(sample_reshaped))]

In [14]:
print(predict('./data_section_5.3/cat/1a9afd33_nohash_0.wav', model=model))

cat
